In [2]:
import lmdb
file = "data/graphs/PUBCHEM/train.lmdb"
mapsize = 1099511627776

env = lmdb.open(
                file,
                subdir=False,
                readonly=False,
                lock=False,
                readahead=False,
                meminit=False,
                max_readers=1,
                map_size=mapsize
                )

In [13]:
import pickle
for idx in range(1000):
    datapoint_pickled = env.begin().get(f"{idx}".encode("ascii"))
    data = pickle.loads(datapoint_pickled)
    print(idx)


0
1
2
3
4


TypeError: a bytes-like object is required, not 'NoneType'